In [ ]:
import numpy
import pandas as pd
import numpy as np
import copy
import os
from os import path
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization,Dropout,Conv2D,MaxPool2D,add,Input,GlobalAveragePooling2D,concatenate
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import sparse_categorical_crossentropy
import pickle
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ZeroPadding2D, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
import math
import numpy as np
import scipy
import h5py
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization  
from tensorflow.keras.layers import Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from matplotlib.pyplot import imshow
from tensorflow.python.keras.optimizer_v2.gradient_descent import SGD

In [ ]:
def crea(img):
  if(img==0):
    return 0
  elif(img==1):
    return 1
  elif(img==2):
    return 2
  elif(img==3):
    return 3
  elif(img==4):
    return 4
  else:
    return 5

#data=pd.read_csv('C:/D Partition/4th year first term/ci/deep_learning/train.csv')

def create_train_data():
    data = pd.read_csv('../input/ci-sc22-places-and-scene-recognition/train.csv')
    training_data = []
    for img in os.listdir('../input/ci-sc22-places-and-scene-recognition/train_images/train_images'):
        path = os.path.join('../input/ci-sc22-places-and-scene-recognition/train_images/train_images', img)
        imgdata = cv2.imread(path)
        imgdata = imgdata[:, :, [2, 1, 0]]  # Make it RGB
        imgdata = cv2.resize(imgdata, (150, 150))
        training_data.append([imgdata, crea(np.array(data['label'][data['image_name'] == img]))])
    shuffle(training_data)
    return training_data

In [ ]:
from tensorflow.python.keras.utils import np_utils
train_data = create_train_data()
X=[i[0] for i in train_data]
Y=[i[1] for i in train_data]
X1 = np.array(X)
Y1 = np.array(Y)
Y1 = np_utils.to_categorical(Y1, 6)

In [ ]:
datagen = ImageDataGenerator(
    rescale=1/255.,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2)

In [ ]:
train_gen = datagen.flow(X1, Y1, batch_size=32,subset='training')

In [ ]:
valid_gen = datagen.flow(X1, Y1, batch_size=8, subset='validation')

In [ ]:
_input = Input((150,150,3))
conv1  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(_input)
conv2  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(conv1)
pool1  = MaxPooling2D((2, 2))(conv2)

conv3  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(pool1)
conv4  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(conv3)
pool2  = MaxPooling2D((2, 2))(conv4)

conv5  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(pool2)
conv6  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv5)
conv7  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv6)
pool3  = MaxPooling2D((2, 2))(conv7)

conv8  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool3)
conv9  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv8)
conv10 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv9)
pool4  = MaxPooling2D((2, 2))(conv10)

conv11 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool4)
conv12 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv11)
conv13 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv12)
pool5  = MaxPooling2D((2, 2))(conv13)

flat   = Flatten()(pool5)
dense1 = Dense(4096, activation="relu")(flat)
dense2 = Dense(1000, activation="relu")(dense1)
output = Dense(6, activation="softmax")(dense2)

model  = Model(inputs=_input, outputs=output)
model.summary()

In [ ]:
from keras.callbacks import LearningRateScheduler,EarlyStopping
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
epochs = 100
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 20
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = SGD(learning_rate=initial_lrate, momentum=0.9, nesterov=False)
opt = Adam(learning_rate=0.01)

lr_sc = LearningRateScheduler(decay, verbose=1)

model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['accuracy'])

In [ ]:
hist = model.fit_generator(train_gen,validation_data=valid_gen,epochs=15,verbose=1)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_it = test_datagen.flow_from_directory('../input/ci-sc22-places-and-scene-recognition/test_images',classes=['test_images'], target_size = (150, 150), class_mode=None, batch_size=1, seed=0,shuffle=False)

In [ ]:
test_files=test_it.filenames

In [ ]:
length=len(test_files)
test_batch_size=sorted([int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=80],reverse=True)[0]  
test_steps=int(length/test_batch_size)
print ( 'test batch size: ' ,test_batch_size, '  test steps: ', test_steps)

In [ ]:
preds=model.predict(test_it, batch_size=test_batch_size, steps=test_steps)

In [ ]:
pr1 = np.argmax(preds,axis=1)

In [ ]:
names = []
for filename in test_files:
    n = filename.split('/')[1]
    names.append(n)

In [ ]:
Fseries=pd.Series(names, name='image_name')
Lseries=pd.Series(pr1, name='label')
predictions_df= pd.concat([Fseries, Lseries], axis=1)
print (predictions_df.head())

In [ ]:
predictions_df.to_csv("submission.csv", index=False)